## CS431/631 Data Intensive Distributed Analytics
### Winter 2020 - Assignment 3
---

**Please edit this (text) cell to provide your name and UW student ID number!**
* **Name:** _Kamran Karim_
* **ID:** _20594936_

---
#### Overview
For this assignment, you will be using Python and Spark to perform some graph analysis, using a graph of the Gnutella server network.   In this graph, each node represents a server, and each (directed) edge represents a connection between servers in Gnutella's peer-to-peer network.  The input file for this assignment, `p2p-Gnutella08-adj.txt`, represents the graph as an adjacency list.   Each server (node) is identified by a unique number, and each line in the file gives the adjacency list for a single server.
For example, this line:
> 91	243	1923	2194

gives the adjacency list for server `91`.   It indicates that there are edges from server `91` to servers `243`, `1923`, and `2194`.    According to the Stanford Network Analysis Project, which collected these data, [the graph includes 6301 servers and 20777 edges](http://snap.stanford.edu/data/p2p-Gnutella08.html).

As you know from the previous assignment, you must tell the Python interpreter where to find Spark before performing any Spark operations in your program.   So, start by doing that:

In [1]:
import findspark
findspark.init("/u/cs451/packages/spark")

from pyspark import SparkContext, SparkConf

and then create a `SparkContext`:

In [2]:
import random
sc = SparkContext(appName="YourTest", master="local[2]", conf=SparkConf().set('spark.ui.port', random.randrange(4000,5000)))

---
### Important

###### The questions that follow ask you to implement functions whose prototypes are given to you. Do **NOT** change the prototypes of the functions. Do **NOT** write code outside of functions. All necessary code should be included in the function body (except for import statements). You may declare functions inside of the function body. When marking, we will execute your code by calling the functions from an external program, which is why your code cannot rely on statements running outside functions. Please remove any call to the functions that you may have introduced for test purposes before submitting your notebook.

---
#### Question 1  (6/24 marks):

To get warmed up, you should first write Spark code to confirm or determine some basic properties of the Gnutella graph.  Write code in the provided functions that will return answers to the following questions, as specified in each function's documentation:
- How many nodes and edges are there in the graph?  (This should confirm the numbers given above.)
- How many nodes of each outdegree are there? That is, how many nodes have no outgoing edges, how many have one outgoing edge, how many have two outgoing edges, and so on?
- How many nodes of each indegree are there?

You should use Spark to answer these questions.   Do *not* load the entire graph into your Python driver program.

In [3]:
file = "p2p-Gnutella08-adj.txt"

In [4]:

def num_nodes_edges():
    """Returns a tuple (num_nodes, num_edges)"""
    a = sc.textFile(file).map(lambda c:(str(c.encode() )[2:].split("\\t")[0].replace("'",""),str(c.encode() )[5:].split("\\t")[1:]))
    nodes = a.count()
    b = a.map(lambda x: ("dummy", len(x[1])))
    edges = b.reduceByKey(lambda x,y: x+y).collect()[0][1]
    return (nodes,edges)
              
    
    
def out_counts():
    """Returns a dictionary where the keys are the outdegrees, and the 
    values are the number of nodes of the corresponding outdegree """
    #### Your code for Question 1.2 should go here
    a = sc.textFile(file).map(lambda c:(str(c.encode() )[2:].split("\\t")[0].replace("'",""),str(c.encode() )[5:].split("\\t")[1:]))
    b = a.map(lambda x: (x[0], len(x[1]))).map(lambda x:(x[1],1)).reduceByKey(lambda x,y: x+y).map(lambda x:(x[1],x[0])).sortByKey(False).map(lambda x:(x[1],x[0]))
    return b.collect()

def in_counts():
    """Returns a dictionary where the keys are the indegrees, and the 
    values are the number of nodes of the corresponding indegree """
    a = sc.textFile(file).map(lambda c:(str(c.encode() )[2:].split("\\t")[0].replace("'",""),str(c.encode() )[5:])).map(lambda x: x[1])
    b = a.flatMap(lambda x:x.split("\\t")).map(lambda x: (x.replace("'",""),1)).reduceByKey(lambda x,y: x+y)
    c = b.map(lambda x:(x[1],1)).reduceByKey(lambda x,y: x+y).map(lambda x:(x[1],x[0])).sortByKey(False).map(lambda x:(x[1],x[0]))
    return c.collect()
print(num_nodes_edges())
print(out_counts())
print(in_counts())

(6301, 20726)
[(0, 3840), (10, 1497), (9, 398), (1, 291), (5, 107), (8, 52), (2, 30), (4, 28), (3, 13), (6, 11), (7, 8), (17, 4), (12, 3), (14, 2), (18, 2), (13, 2), (34, 1), (24, 1), (22, 1), (28, 1), (46, 1), (48, 1), (25, 1), (29, 1), (19, 1), (47, 1), (11, 1), (31, 1), (41, 1)]
[(1, 2494), (2, 1281), (3, 874), (4, 554), (5, 333), (6, 229), (7, 141), (8, 79), (9, 68), (10, 37), (11, 27), (12, 24), (13, 19), (14, 14), (15, 6), (20, 4), (81, 4), (70, 3), (60, 3), (531, 3), (71, 3), (67, 3), (604, 2), (32, 2), (62, 2), (18, 2), (72, 2), (56, 2), (66, 2), (19, 2), (21, 2), (69, 2), (31, 2), (50, 1), (82, 1), (30, 1), (46, 1), (16, 1), (536, 1), (580, 1), (52, 1), (76, 1), (38, 1), (86, 1), (44, 1), (22, 1), (78, 1), (589, 1), (579, 1), (949, 1), (53, 1), (57, 1), (47, 1), (25, 1), (33, 1), (603, 1), (85, 1), (83, 1), (87, 1), (61, 1), (27, 1), (73, 1), (35, 1), (63, 1), (79, 1), (55, 1), (41, 1), (91, 1), (23, 1)]


---
Your main objective for this assignment is to perform *single source personalized page rank* over the Gnutella graph.  To get started, you should make sure that you have a clear understanding of ordinary (i.e., non-personalized) page rank.  Read the description of page rank in Section 5.3 of [the course textbook](http://mapreduce.cc/).   Personalized page rank is like ordinary page rank except:
- One node in the graph is designated as the *source* node. Personalized page rank is performed with respect to that source node.
- Personalized page rank is initialized by assigning all probability mass to the source node, and none to the other nodes. In contrast, ordinary page rank is initialized by giving all nodes the same probability mass.
- Whenever personalized page rank makes a random jump, it jumps back to the source node. In contrast, ordinary page rank may jump to any node.
- In personalized page rank, all probability mass lost dangling nodes is put back into the source nodes.  In ordinary page rank, lost mass is distributed evenly over all nodes.

#### Question 2  (10/24 marks):

Your task is to write a Spark program to perform personalized page rank over the Gnutella graph for a specified number of iterations, and of course a specific node. The function you will implement takes three input values:
- source node id (a positive integer)
- iteration count (a positive integer)
- random jump factor value (a float between 0 and 1)

The function should perform personalized page rank, with respect to the specified source node, over the Gnutella graph, for the specified number of iterations, using Spark.
The output of your function should be a list of the 10 nodes with the highest personalized page rank with respect to the given source. For each of the 10 nodes, return the node's id and page rank value as a tuple. The list returned by the function should therefore look something like this: `[(node_id_1, highest_pagerank_value), ..., (node_id_10, 10th_highest_pagerank_value)]`

In [9]:
import numpy as np
import random as rd
file = "p2p-Gnutella08-adj.txt"


In [63]:
def pair(s):
    l = list()
    tot = len(s)-1
    for i in s:
        j = i.replace("'","")
        k = s[0].replace("'","")
        if tot == 0:
            l.append((k,[],0))
        elif i != s[0]:
            l.append((k,j, tot))
    return l
def swap(x):
    return (x[1],x[0])
def add(x,y):
    return x+y
original =  sc.textFile(file).map(lambda c:str(c.encode() )[2:].split("\\t")).flatMap(pair)
orig = original.map(lambda x: (x[0],(x[1],x[2])))

In [105]:
def personalized_page_rank(s_n_id, iter, j_fac):
    """Returns a list of the 10 nodes with the highest page rank value along with their value, as tuples
    [(node_id_1, highest_pagerank_value), ..., (node_id_10, 10th_highest_pagerank_value)]"""
    n_j = 1-j_fac
    try:
        original
    except NameError:
        original = None
        orig = None
    
    if original is None:
        original =  sc.textFile(file).map(lambda c:str(c.encode() )[2:].split("\\t")).flatMap(pair)
        original.cache()
        orig = original.map(lambda x: (x[0],(x[1],x[2])))
        orig.cache()

    try:
        int(s_n_id)
    except:
        s_n_id = -1
 
    if int(s_n_id) in range(original.count() +1) and isinstance(iter, int) and iter>0 and (0<= j_fac <= 1): 
        if type(s_n_id) is not str:
            s_n_id = str(s_n_id)
        
        iterations = original.map(lambda x:(x[0], x[1],x[2], 1 if x[0] == s_n_id  else 0))
       
        for i in range(iter):
            rank = iterations.map(lambda x : (x[1],n_j* x[3]/x[2], j_fac*x[3]/x[2]) if x[2] != 0 else  (s_n_id, x[3], 0))
            s_n_add = rank.map(lambda x: ("k", x[2])).reduceByKey(lambda x,y:x+y).collect()[0][1]
            rank = rank.map(lambda x: (x[0], x[1])).reduceByKey(lambda x,y:x+y).map(lambda x : (x[0], x[1]+s_n_add) if x[0] == s_n_id else (x[0], x[1]))
            iterations = orig.leftOuterJoin(rank).map(lambda x: (x[0],x[1][0][0],x[1][0][1],x[1][1] if x[1][1] is not None else 0 ))
        
        normal = rank.map(lambda x: ("n", x[1])).reduceByKey(add).collect()[0][1]
        
        return rank.map(lambda x: (x[0], x[1]/normal)).map(swap).sortByKey(False).map(swap).take(10)
    else:
        return "s_n_id has to one of the nodes, iter has to be a natural number (and possibly small if you wanna finish the computation today) and factor has to be between 0 and 1"
       

In [108]:
#test the function here
s_node = 
n_iterations = 
jump_factor = 
personalized_page_rank(s_node,n_iterations , jump_factor)


[('3', 0.4796339200000007),
 ('1860', 0.03742151111111117),
 ('1896', 0.03202560000000006),
 ('1898', 0.031974400000000056),
 ('826', 0.031974400000000056),
 ('1895', 0.031974400000000056),
 ('703', 0.031974400000000056),
 ('1897', 0.031974400000000056),
 ('1287', 0.031974400000000056),
 ('1097', 0.031974400000000056)]

---
#### Question 3  (4/24 marks):

For the previous question, you implemented personalized page rank that ran for a specified number of iterations.  However, it is also common to write iterative algorithms that run until some specified termination condition is reached.
For example, for page rank, suppose the $p_i(x)$ represents the probability mass assigned to node $x$ after the $i$th iteration of the algorithm.  ($p_0(x)$ is the initial probability mass of node $x$.)   We define the change of $x$'s probability mass on the $i$th iteration as $\lvert p_i(x)-p_{i-1}(x) \rvert$.   Then, we can iterate personalized page rank until the maximum (over all nodes) change is less than a specified threshold, i.e, until all nodes' page ranks have converged.

For this question, modify your personalized page rank implementation from Question 2 so that it iterates until the 
maximum node change is less than $\frac{0.5}{N}$, where $N$ represents the number of nodes in the graph.
This version of the function should take only two inputs: the source node id and the random jump factor.

If you were unable to get personalized page rank working in Question 2, replace the code cell below with a text (Markdown) cell, and briefly describe how you *would* have modified your program to incorporate a termination condition based on maximum node change, for partial credit. 

In [103]:
def personalized_page_rank_stopping_criterion(s_n_id, j_fac):
    """Returns a list of the 10 nodes with the highest page rank value along with their value, as tuples
    [(node_id_1, highest_pagerank_value), ..., (node_id_10, 10th_highest_pagerank_value)]"""
    if type(s_n_id) is not str:
        s_n_id = str(s_n_id)
    nodes = num_nodes_edges()[0]
    T =0.5/nodes
    
    try:
        original
    except NameError:
        original = None
        orig = None
    if original is None:
        original =  sc.textFile(file).map(lambda c:str(c.encode() )[2:].split("\\t")).flatMap(pair)
        orig = original.map(lambda x: (x[0],(x[1],x[2])))
        orig.cache()
        original.cache()
    try:
        int(s_n_id)
    except:
        s_n_id = -1
    if int(s_n_id) in range(original.count() +1) and (0<= j_fac <= 1):  

        iterations = original.map(lambda x:(x[0], x[1],x[2], 1 if x[0] == s_n_id  else 0))
        dummy =0
        iter = 1
        n = 0
        while iter>T:
            rank = iterations.map(lambda x : (x[1],n_j* x[3]/x[2], j_fac*x[3]/x[2]) if x[2] != 0 else  (s_n_id, x[3], 0))
            s_n_add = rank.map(lambda x: ("k", x[2])).reduceByKey(add).collect()[0][1]
            rank = rank.map(lambda x: (x[0], x[1])).reduceByKey(add).map(lambda x : (x[0], x[1]+s_n_add) if x[0] == s_n_id else (x[0], x[1]))
            iterations = orig.leftOuterJoin(rank).map(lambda x: (x[0],x[1][0][0],x[1][0][1],x[1][1] if x[1][1] is not None else 0 ))
            if dummy == 1:
                iter = rank.join(rank2).map(lambda x :  abs(x[1][0]-x[1][1])).max()
            dummy = 1
            rank2 = rank
            n+=1
        normal = rank.map(lambda x: ("n", x[1])).reduceByKey(add).collect()[0][1]
        rank = rank.map(lambda x: (x[0], x[1]/normal)).map(swap).sortByKey(False).map(swap)

        return rank.take(10)
    else:
        return "s_n_id has to one of the nodes and factor has to be between 0 and 1"




In [104]:
#test the function here
s_node =
jump_factor = 
personalized_page_rank_stopping_criterion(s_node, jump_factor)


---
#### Question 4  (4/24 marks):

Spark provides the ability to *cache* RDDs.   This is useful when an RDD will be used more than once.   Instead of computing the same RDD multiple times, it can be computed once, cached, and then re-used from the cache.   Read about caching in the Spark textbook, or in the [persistence section of the Spark RDD programming guide](https://spark.apache.org/docs/latest/rdd-programming-guide.html#rdd-persistence).   Caching can be particularly effective for iterative Spark applications, like those you are writing for this assignment.

For this question, go back to the code that you wrote to answer Question 3, and add caching.   (Caching will not affect the functionality of your code, i.e., what it computes.   It should only affect performance.)   Don't worry about different persistence levels.   Just use `cache()`, which caches RDDs at the default persistence level.

In addition to adding `cache()` calls in your code, use the text cell below to briefly explain how you decided which RDDs to cache. 

If you were not able to finish Question 3, add caching annotations to your solution for Question 2 instead.

---
#### Your answer to Question 4:

*In my code the two rdds that are being reused are named as original and orig. Now, ideally we will cache these rdds outside the function, as for any function call, we are using the same rdd. But as I have been losing marks in the previous assignment for doing any operations outside the function, I would rather be safe than sorry.
So the way I cache the rdd in the function is first check if the original rdd already exists, if not, than create it and cache() it, similar with orig rdd. Since all other rdds are being updated in each iterations, then it would not make sense to cache any of those files.*




---
Don't forget to save your workbook!   When you are finished and you are ready to submit your assignment, download your notebook file (.ipynb) from the hub to your machine, and then follow the submission instructions in the assignment.